In [17]:
# -*- coding: utf-8 -*-
#this notebook
#1.get a start and end date
#2.for each date
    #1.retrieves the table ga_sessions in 3 suisses (specify account)
    #2.transforms it (selects and renames columns)
    #3.export it to gs

#importing libraries
from google.cloud import bigquery, storage
from datetime import datetime,date, timedelta

#getting today's date and time
now = datetime.now()
today_date_hour=now.strftime("%Y-%m-%d %H:%M")
#getting yesterday's date
today_datetime=date.today()
today=today_datetime.strftime('%Y%m%d')
yesterday_datetime= (date.today() - timedelta(1))
yesterday=yesterday_datetime.strftime('%Y%m%d')

#start and end date
#if you want to retrieve a history, replace start date by date(2018,5,20) for instance
start_date=yesterday_datetime
end_date=yesterday_datetime

#params
project_id='ringed-cell-652'
    #input
account_id = '72729485'
dataset_id_original=account_id
    #output
dataset_id_transformed=account_id
    #gs
bucket_name='{}_users'.format(account_id)


#creating clients with auth keys
service_account_file='./../02_gcs_access/3Suisses GAP-9a0d1d5296de.json'

bigquery_client=bigquery.Client(project=project_id).from_service_account_json(service_account_file)
storage_client=storage.Client('ringed-cell-652').from_service_account_json(service_account_file)


def main_scheduler():

    #launching process
    d=start_date
    while d <= end_date:
        print ('\n\n date : ',d)
        d_suffix=d.strftime('%Y%m%d')
        tranforming_table_in_bq(d_suffix)
        exporting_to_gs(d_suffix)
        downloading_to_local(d_suffix)
        
        #itération sur la date
        delta=timedelta(days=1)
        d += delta
        
    print ('\n\nthe whole time period has been processed')


def tranforming_table_in_bq(d_suffix):
    
    #table names changing depending on date
    table_id_original='ga_sessions_{}'.format(d_suffix)
    table_id_transformed='users_{}'.format(d_suffix)

    #2.setting up configs
        #configs
    job_config=bigquery.QueryJobConfig()
        #sql_query
    sql_query='''
    #standardSQL
    CREATE OR REPLACE TABLE `dataset_id_transformed.table_id_transformed` AS
      (
      SELECT
      fullvisitorId as NUMCLI,
      visitId as NUMVSTWEB,
      date as DATVSTWEB,
      totals.timeOnSite as DURVSTWEBSEC,
      totals.pageviews as NBRPAGVST

      FROM
      `project_id.dataset_id_original.table_id_original`
      )

    '''
    sql_query=sql_query.replace(
        'project_id',project_id).replace(
        'dataset_id_original',dataset_id_original).replace(
        'dataset_id_transformed',dataset_id_transformed).replace(
        'table_id_transformed',table_id_transformed).replace(
        'table_id_original',table_id_original)

    #3.launching job
    query_job=bigquery_client.query(
    sql_query,
    job_config
    )
    #print starting query job
    print ('---------01. starting query job')

    #4.checking errors
        #wait for results
    query_job.result()
        #checking errors
    print ('job_errors = ', query_job.errors)
        #asserting job state is done
    assert query_job.state == 'DONE'
    print ('query_job = ',query_job.state)
    print ('---------01. finished query job')
    
    return(table_id_original,table_id_transformed)
    
def exporting_to_gs(d_suffix):
    #2.setting up configs
    #inputs 
    dataset_id=dataset_id_transformed
    dataset_ref=bigquery_client.dataset(dataset_id)
    table_id='users_{}'.format(d_suffix)
    table_ref=dataset_ref.table(table_id)
    
    
    #outputs
    file_name='{}_{}.csv'.format(dataset_id_original,table_id)
    destination_uri='gs://{}/{}'.format(bucket_name,file_name)
    #configs
    job_config=bigquery.ExtractJobConfig()
    job_config.compression=True

    #3.launching job
    extract_job=bigquery_client.extract_table(
    table_ref,
    destination_uri,
    job_config=job_config)
    #print starting extracting job
    print ('---------02. starting extracting job')

    #4.checking errors
        #wait for results
    extract_job.result()
        #checking errors
    print ('job_errors = ', extract_job.errors)
        #asserting job state is done
    assert extract_job.state == 'DONE'
    print ('extract_job = ',extract_job.state)
    print ('---------02. finished extracting job')

def downloading_to_local(d_suffix):
    #getting the desired file
    bucket=storage_client.get_bucket(bucket_name)
    file_to_download='{}_{}.csv'.format(bucket_name,d_suffix)
    blob=bucket.blob(file_to_download)

    #downloading the file to local temporary file
    print ('---------03. starting downloading_to_local job')
    temporary_file='../03_file_exports/temp_file.csv'
    blob.download_to_filename(temporary_file)
    print('---------03. downloaded %s from gcs to %s'%(file_to_download,temporary_file))
 
    
if __name__=='__main__':
    main_scheduler()



 date :  2018-06-04
---------01. starting query job
job_errors =  None
query_job =  DONE
---------01. finished query job
---------02. starting extracting job
job_errors =  None
extract_job =  DONE
---------02. finished extracting job
---------03. starting downloading_to_local job
---------03. downloaded 72729485_users_20180604.csv from gcs to ../03_file_exports/temp_file.csv


the whole time period has been processed


['3Suisses GAP-9a0d1d5296de.json']